## **L14_model evaluation (prática)**

1 - Utilizando os dados "diabetes" faça

1.1 - Criar um pipeline com um pré-processamento e classificador a sua escolha

1.2 - Faça um gridsearch para achar os melhores hyperparâmetros do método

1.3 - Estime, em cima da base de dados de teste, a acurácia final, dada a melhor combinação de hyperparâmetros possíveis

1.4 - Calcule o erro (01 loss no caso de classificação)

1.5 - Decomponha o erro em termos do seu bias e variância

1.4 - Verifique a existência de overfitting e underfitting, justifique (olhar e analisar taxas de acerto do treino, teste, bias e variância). A argumentação de ter ou não overfitting e underfitting deve ser feita com comentários no código.

1.5 - Compartilhar o link (no colab, botão superior direito, compartilhar, alterar opção para "qualquer um com link pode visualizar", copiar link, e colar o link aqui para verificação)

In [ ]:
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import zero_one_loss
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split



In [ ]:
pip install mlxtend --upgrade --no-deps

     |████████████████████████████████| 1.3 MB 4.2 MB/s 
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [ ]:
from mlxtend.evaluate import bias_variance_decomp

diabetes = datasets.load_diabetes()
X = diabetes.data
y = diabetes.target

In [ ]:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1)

##Pipeline

In [ ]:
pipe = Pipeline([
        ('normalizador', StandardScaler()),
        ('redutor_dim', PCA()),
        ('classificador', DecisionTreeClassifier())
])

pipe.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('normalizador',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('redutor_dim',
                 PCA(copy=True, iterated_power='auto', n_components=None,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('classificador',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features=None, max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        presort='deprecated', random_state=None,
                                

##Grid-Search

In [ ]:
params = {
    'redutor_dim__n_components': [1, 2, 3, 4, 5, 6, 7],
    'classificador__max_depth':[2,3,4,5,6,7,8,9],
    'classificador__criterion':['gini','entropy'],
   
}

grid = GridSearchCV(pipe, cv=2, param_grid= params, scoring='accuracy' )

In [ ]:
grid.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  % (min_groups, self.n_splits)), UserWarning)


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('normalizador',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('redutor_dim',
                                        PCA(copy=True, iterated_power='auto',
                                            n_components=None,
                                            random_state=None,
                                            svd_solver='auto', tol=0.0,
                                            whiten=False)),
                                       ('classificador',
                                        DecisionTreeClassifier(ccp_alpha=0.0,
                                                               class_weight=None,
                                              

##Predict

In [ ]:
y_test_pred = grid.predict(X_test)

print(grid.best_params_,grid.best_score_)

{'classificador__criterion': 'entropy', 'classificador__max_depth': 2, 'redutor_dim__n_components': 3} 0.019838212634822806


In [ ]:
grid_r = grid.best_estimator_
print(grid_r)

Pipeline(memory=None,
         steps=[('normalizador',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('redutor_dim',
                 PCA(copy=True, iterated_power='auto', n_components=3,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('classificador',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='entropy', max_depth=2,
                                        max_features=None, max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        presort='deprecated', random_state=None,
                                   

In [ ]:
accuracy_score(y_test, y_test_pred) #Resultado

0.011235955056179775

In [ ]:
zero_one_loss(y_test,y_test_pred, normalize=False)

88

In [ ]:
mean_squared_error(y_test, y_test_pred) #Erro

8129.224719101124

In [ ]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
    grid_r, X_train, y_train, X_test, y_test, 
    loss='mse', num_rounds=3,
    random_seed=1)

print('Avarage expected loss: %.3f' %avg_expected_loss)
print('Avarage bias^2: %.3f' %avg_bias)
print('Avarage variance: %.3f' %avg_var)

Avarage expected loss: 7937.536
Avarage bias^2: 5491.813
Avarage variance: 2445.723


Resultado: Por ter um alto bias, é um caso de underfiting